# Data collection from spingboard for Country Club case study

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

In [1]:
""" import packages """

import pandas as pd
import numpy as np
import sqlite3 as sql

In [2]:
database = "sqlite_db_pythonsqlite.db"

con = sql.connect(database)

In [3]:
""" check the structure of database """

# check the table names
cursor = con.cursor()
cursor.execute('SELECT name from sqlite_master where type= "table"')
table_names = (cursor.fetchall())

# check the columns for each table
for itab in table_names:
    print("--- table %s ---"%itab)
    
    cursor2 = con.execute('select * from %s'%itab)
    colnames = cursor2.description
    for row in colnames:
        print(row[0]) 

--- table Bookings ---
bookid
facid
memid
starttime
slots
--- table Facilities ---
facid
name
membercost
guestcost
initialoutlay
monthlymaintenance
--- table Members ---
memid
surname
firstname
address
zipcode
telephone
recommendedby
joindate


### Q10: 
Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [4]:
query = '''
with sub as (
select fac.name as Facilities,
    case when mem.memid = 0 then fac.guestcost*bk.slots
         else fac.membercost*bk.slots 
         end as book_cost
from Bookings as bk
inner join Facilities as fac using (facid)
inner join Members as mem using (memid)
)
select Facilities, sum(book_cost) as "Total Revenue"
from sub
group by Facilities
having "Total Revenue" < 1000
order by "Total Revenue" desc;
'''

q_df = pd.read_sql_query(query, con)
q_df

,Facilities,Total Revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


### Q11: 
Produce a report of members and who recommended them in alphabetic surname, firstname order

In [5]:
query = '''
select m1.surname || ", " || m1.firstname as Member,
       m2.surname || ", " || m1.firstname as "Recommended by"
from Members as m1
-- used left join to report the entire member including ones who were not recommended by other member
left join Members as m2 on m1.recommendedby = m2.memid
-- remove the guest member
where m1.surname not like "GUEST%"
order by m1.surname, m1.firstname;
'''

q_df = pd.read_sql_query(query, con)
q_df

,Member,Recommended by
0,"Bader, Florence","Stibbons, Florence"
1,"Baker, Anne","Stibbons, Anne"
2,"Baker, Timothy","Farrell, Timothy"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Gerald"
5,"Coplin, Joan","Baker, Joan"
6,"Crumpet, Erica","Smith, Erica"
7,"Dare, Nancy","Joplette, Nancy"
8,"Farrell, David",None
9,"Farrell, Jemima",None


**note: concat() function doesn't work with sqlite! Instead, it uses '||' operator to combine strings.**

### Q12: 
Find the facilities with their usage by member, but not guests

In [6]:
query = '''
with sub as(
    select slots, 
           memid,
           facid,
           sum(slots) over() as total_slots
    from Bookings
    where memid != 0
)

select fac.name as Facility,
    round(sum(sub.slots * 100. / sub.total_slots), 2) as Usage
from sub
left join Facilities as fac using (facid)
where sub.memid != 0
group by fac.name
order by usage desc;
'''

q_df = pd.read_sql_query(query, con)
q_df

,Facility,Usage
0,Badminton Court,15.99
1,Tennis Court 1,14.09
2,Massage Room 1,13.02
3,Tennis Court 2,12.99
4,Snooker Table,12.66
5,Pool Table,12.60
6,Table Tennis,11.69
7,Squash Court,6.16
8,Massage Room 2,0.80


### Q13: 
Find the facilities usage by month, but not guests

In [7]:
query = '''
with sub as(
    select slots, 
           memid,
           facid,
           starttime,
           sum(slots) over() as total_slots
    from Bookings
    where memid != 0
)

select strftime('%m', sub.starttime) as Month,
       fac.name as Facility,
       round(sum(sub.slots * 100. / sub.total_slots),2) as Usage
from sub
left join Facilities as fac using (facid)
where sub.memid != 0
group by Month, Facility
order by Month, Usage desc;
'''

q_df = pd.read_sql_query(query, con)
q_df

,Month,Facility,Usage
0,07,Tennis Court 1,2.96
1,07,Massage Room 1,2.44
2,07,Badminton Court,2.43
3,07,Snooker Table,2.06
4,07,Tennis Court 2,1.81
5,07,Pool Table,1.62
6,07,Table Tennis,1.44
7,07,Squash Court,0.74
8,07,Massage Room 2,0.12
9,08,Badminton Court,6.10


**note: extract(Month from starttime) does not work in sqlite3. Instead it should use strftime("%m", starttime)**